In [1]:
import duckdb
import pandas as pd

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:///:memory:

### Read data

In [ ]:
# dataset_path = 'C:/Users/Cristian Burich/Desktop/MA/segundo/eyf/datasets/' # '/content/drive/MyDrive/Colab Notebooks/DMEyF/2024/datos/'
# dataset_file = 'competencia_02_crudo.csv'

dataset_path = '/home/cburich_pymnts/buckets/b1/datasets/' 
dataset_file = 'competencia_03.parquet.csv'

In [ ]:
# data = pd.read_csv(dataset_path+dataset_file)
# data.head(10)

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,Visa_madelantodolares,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo
0,249221109,201901,1,0,1,59,276,7597.55,47433.58,5654.59,...,0.0,1.0,0.0,-44919.57,3.23,7136.0,24336.99,13.0,0.0,1466.25
1,249221468,201901,1,0,1,51,13,738.16,-325.09,491.31,...,0.0,1.0,0.0,-30184.75,15.24,393.0,16345.79,8.0,0.0,1548.36
2,249223005,201901,1,0,1,46,178,1014.31,9434.15,417.36,...,0.0,1.0,0.0,-3730.14,0.00,2755.0,1181.65,3.0,0.0,3917.82
3,249228180,201901,1,0,1,64,296,1028.33,6873.80,1129.43,...,0.0,1.0,0.0,-13196.25,0.00,8194.0,1892.04,1.0,0.0,2017.56
4,249232117,201901,1,0,1,77,349,11617.61,43291.55,10726.08,...,0.0,1.0,0.0,-22073.65,0.00,7876.0,21902.85,6.0,0.0,891.48
5,249236712,201901,1,0,1,42,1,1746.17,1746.17,0.00,...,NaN,1.0,0.0,NaN,NaN,10.0,NaN,NaN,NaN,0.00
6,249237079,201901,1,0,1,62,117,19193.25,110961.55,3147.21,...,0.0,8.0,0.0,-3688.58,0.00,3531.0,257.94,1.0,0.0,586.50
7,249237446,201901,1,0,1,52,253,3268.57,20250.17,220.94,...,0.0,8.0,0.0,-5574.87,0.00,3222.0,-1.34,4.0,0.0,680.34
8,249239632,201901,1,0,1,50,239,990.51,8868.52,343.85,...,0.0,1.0,0.0,-18110.05,0.00,5054.0,1596.32,2.0,0.0,1114.35
9,249242331,201901,1,0,1,53,256,3429.46,27997.33,1153.68,...,0.0,1.0,0.0,-49544.20,116.27,5708.0,35766.21,18.0,0.0,2498.49


In [ ]:
%%sql

create or replace table competencia_03 as
select *
from read_parquet("{{dataset_path + dataset_file}}");

,Success


In [ ]:
%%sql
select
    foto_mes
    , count(*) as cantidad # cuenta cuantos casos hay en cada foto_mes
                           # y lo guarda en un campo llamado cantidad
from competencia_03
group by foto_mes

,foto_mes,cantidad
0,201901,124752
1,201902,125799
2,201903,126436
3,201904,126996
4,201905,127659
5,201906,129186
6,201907,130724
7,201908,132664
8,201909,134314
9,201910,136682


In [ ]:
# Vamos a usar TODOS los datos

# %%sql

# create or replace table competencia_03 as
# select *
# from competencia_03
# where foto_mes = 202101
# or foto_mes = 202102
# or foto_mes = 202103
# or foto_mes = 202104
# or foto_mes = 202105
# or foto_mes = 202106
# or foto_mes = 202107
# or foto_mes = 202108

,Success


In [ ]:
%%sql

create or replace table competencia_03 as

with periodos as (
    select distinct foto_mes from competencia_03
), clientes as (
    select distinct numero_de_cliente from competencia_03
), todo as (
    select numero_de_cliente, foto_mes from clientes cross join periodos
), clase_ternaria as (
    select
        c.*
        , if(c.numero_de_cliente is null, 0, 1) as mes_0 # genero la columna mes_0 y sobre ella genero mes_1 y mes_2
        , lead(mes_0, 1) over (partition by t.numero_de_cliente order by foto_mes) as mes_1
        , lead(mes_0, 2) over (partition by t.numero_de_cliente order by foto_mes) as mes_2
        , if(mes_2 = 1, 'CONTINUA',
             if(mes_1 = 0, 'BAJA+1',
                if(mes_2 = 0,'BAJA+2',NULL))) as clase_ternaria # Replazar null por la lógica que genera el target
    from todo t
    left join competencia_03 c using (numero_de_cliente, foto_mes)
) 

select * EXCLUDE (mes_0, mes_1, mes_2) # descarto las columnas auxiliares
from clase_ternaria
where mes_0 = 1 # descarto las filas de los clientes para los meses previos a que ingresaran al banco.

,Success


In [ ]:
%sql select count(*) from competencia_03


,count_star()
0,1312540


In [ ]:
%%sql
PIVOT competencia_03
on clase_ternaria
USING count(numero_de_cliente)
GROUP BY foto_mes

,foto_mes,BAJA+1,BAJA+2,CONTINUA
0,202101,635,785,160606
1,202102,785,1017,160844
2,202103,1020,981,161684
3,202104,982,1189,161919
4,202105,1189,911,162523
5,202106,908,1074,162894
6,202107,1075,0,0
7,202108,0,0,0


In [ ]:
%%sql 

select * from competencia_03

limit 10

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo,clase_ternaria
0,249461691,202101,1,0,0,63,320,5184.37,47486.85,2964.22,...,4,0.0,-39809.64,0.0,7552,25839.92,16.0,0.0,2123.13,CONTINUA
1,249461691,202102,1,0,0,64,321,4427.14,48887.16,2984.35,...,-3,0.0,-32646.53,0.0,7580,41661.42,20.0,0.0,2944.23,CONTINUA
2,249461691,202103,1,0,0,64,322,4935.91,50391.10,3254.86,...,1,0.0,-48657.86,0.0,7611,25120.08,15.0,0.0,2052.75,CONTINUA
3,249461691,202104,1,0,0,64,323,3801.55,50742.98,1929.76,...,2,0.0,-31233.89,0.0,7641,26204.83,15.0,0.0,2158.32,CONTINUA
4,249461691,202105,1,0,0,64,324,4234.54,51412.66,1614.34,...,5,0.0,-33222.30,0.0,7672,21115.32,15.0,0.0,1982.37,CONTINUA
5,249461691,202106,1,0,0,64,325,6527.06,54176.74,3565.12,...,0,0.0,-29974.72,0.0,7702,38276.76,17.0,0.0,2756.55,CONTINUA
6,249461691,202107,1,0,0,64,326,7083.31,57147.29,3713.74,...,3,0.0,-44887.16,0.0,7733,30133.20,13.0,0.0,2615.79,None
7,249461691,202108,1,0,0,64,327,7988.86,61521.60,3975.73,...,6,0.0,-42319.34,0.0,7764,29601.25,16.0,0.0,2803.47,None
8,251867778,202101,1,0,0,58,261,4870.53,41960.19,2450.71,...,4,0.0,NaN,NaN,5653,NaN,NaN,NaN,0.00,CONTINUA
9,251867778,202102,1,0,0,58,262,5025.90,44585.85,2448.21,...,-3,0.0,NaN,NaN,5681,NaN,NaN,NaN,0.00,CONTINUA


### Elimino variables con data drifting / concept drifting

In [ ]:
%%sql
select cprestamos_personales, mprestamos_personales
from competencia_03
limit 5

,cprestamos_personales,mprestamos_personales
0,0,0.0
1,0,0.0
2,0,0.0
3,0,0.0
4,0,0.0


In [ ]:
%%sql
create or replace table competencia_03 as
select * exclude(cprestamos_personales,mprestamos_personales)
from competencia_03

,Success


### Sumas de columnas

In [ ]:
%%sql
create or replace table competencia_03 as
select
    *
    , mcuenta_corriente_adicional + mcuenta_corriente + mcaja_ahorro_adicional + mcaja_ahorro_dolares + mcaja_ahorro AS suma_alt # ninguna tiene null
    , cseguro_vida + cseguro_auto +	cseguro_vivienda + cseguro_accidentes_personales AS cseguro_todos # ninguna tiene null
    , ifnull(matm, 0) + ifnull(matm_other, 0) as matm_total    
    , ifnull(Visa_msaldototal, 0) + ifnull(Visa_msaldototal, 0) as tarjeta_msaldototal
    , ifnull(ccuenta_corriente, 0) +  ifnull(ccaja_ahorro, 0) as cc_ca
    , ifnull(ctarjeta_visa, 0) +  ifnull(ctarjeta_Visa, 0) as ctarjeta_total
    , ifnull(ctarjeta_visa_transacciones, 0) +  ifnull(ctarjeta_master_transacciones, 0) as ctarjeta_total_transacciones
    , ifnull(mtarjeta_visa_consumo, 0) +  ifnull(mtarjeta_master_consumo, 0) as mtarjeta_total_consumo
    , ifnull(mplazo_fijo_dolares, 0) +  ifnull(mplazo_fijo_pesos, 0) as mplazo_fijo
    , ifnull(minversion1_pesos, 0) +  ifnull(minversion1_dolares, 0) as minversion1
    , ifnull(mpayroll, 0) +  ifnull(mpayroll2, 0) as mpayroll_todos
    , ifnull(mcuenta_debitos_automaticos, 0) +  ifnull(mtarjeta_total_consumo, 0) as consumos_mas_DA
    , ifnull(ctarjeta_visa_debitos_automaticos, 0) +  ifnull(ctarjeta_master_debitos_automaticos, 0) as ctarjeta_todas_debitos_automaticos
    , ifnull(mttarjeta_visa_debitos_automaticos, 0) +  ifnull(mttarjeta_master_debitos_automaticos, 0) as mtarjeta_todas_debitos_automaticos
    , ifnull(ctarjeta_visa_descuentos, 0) +  ifnull(ctarjeta_master_descuentos, 0) as ctarjeta_todas_descuentos
    , ifnull(mtarjeta_visa_descuentos, 0) +  ifnull(mtarjeta_master_descuentos, 0) as mtarjeta_todas_descuentos
    , ifnull(ccomisiones_mantenimiento, 0) +  ifnull(ccomisiones_otras, 0) as comisiones
    , ifnull(mcomisiones_mantenimiento, 0) +  ifnull(mcomisiones_otras, 0) as mcomisiones_suma
    , ifnull(ctransferencias_recibidas, 0) +  ifnull(ctransferencias_emitidas, 0) as ctransferencias
    , ifnull(mtransferencias_recibidas, 0) +  ifnull(mtransferencias_emitidas, 0) as mtransferencias
    , ifnull(Master_msaldopesos, 0) +  ifnull(Visa_msaldopesos, 0) as tarjetas_msaldopesos
    , ifnull(Master_msaldodolares, 0) +  ifnull(Visa_msaldodolares, 0) as tarjetas_msaldodolares
    , ifnull(Master_mconsumospesos, 0) +  ifnull(Visa_mconsumospesos, 0) as tarjetas_mconsumospesos
    , ifnull(Master_mconsumosdolares, 0) +  ifnull(Visa_mconsumosdolares, 0) as tarjetas_mconsumosdolares
    , ifnull(Master_mlimitecompra, 0) +  ifnull(Visa_mlimitecompra, 0) as tarjetas_mlimitecompra
    , ifnull(Master_madelantopesos, 0) +  ifnull(Visa_madelantopesos, 0) as tarjetas_madelantopesos
    , ifnull(Master_madelantodolares, 0) +  ifnull(Visa_madelantodolares, 0) as tarjetas_madelantodolares
    , ifnull(Master_mpagado, 0) +  ifnull(Visa_mpagado, 0) as tarjetas_mpagado
    , ifnull(Master_mpagospesos, 0) +  ifnull(Visa_mpagospesos, 0) as tarjetas_mpagospesos
    , ifnull(Master_mpagosdolares, 0) +  ifnull(Visa_mpagosdolares, 0) as tarjetas_mpagosdolares
    , ifnull(Master_mconsumototal, 0) +  ifnull(Visa_mconsumototal, 0) as tarjetas_mconsumototal
    , ifnull(Master_cconsumos, 0) +  ifnull(Visa_cconsumos, 0) as tarjetas_cconsumos
    , ifnull(Master_cadelantosefectivo, 0) +  ifnull(Visa_cadelantosefectivo, 0) as tarjetas_cadelantosefectivo
    , ifnull(Master_mpagominimo, 0) +  ifnull(Visa_mpagominimo, 0) as tarjetas_mpagominimo
    , ifnull(Master_mfinanciacion_limite, 0) +  ifnull(Visa_mfinanciacion_limite, 0) as tarjetas_mfinanciacion_limite
from competencia_03

,Success


### Promedios moviles

Variables originales

In [ ]:
# %%sql
# create or replace table competencia_03 as
# select
#     *
#     , avg(mrentabilidad) over ventana_3 as avg3_mrentabilidad
#     , avg(mrentabilidad_annual) over ventana_3 as avg3_mrentabilidad_annual
#     , avg(mcomisiones) over ventana_3 as avg3_mcomisiones
#     , avg(mactivos_margen) over ventana_3 as avg3_mactivos_margen
#     , avg(mpasivos_margen) over ventana_3 as avg3_mpasivos_margen
#     , avg(cproductos) over ventana_3 as avg3_cproductos
#     , avg(tcuentas) over ventana_3 as avg3_tcuentas
#     , avg(ccuenta_corriente) over ventana_3 as avg3_ccuenta_corriente
#     , avg(mcuenta_corriente_adicional) over ventana_3 as avg3_mcuenta_corriente_adicional
#     , avg(mcuenta_corriente) over ventana_3 as avg3_mcuenta_corriente
#     , avg(ccaja_ahorro) over ventana_3 as avg3_ccaja_ahorro
#     , avg(mcaja_ahorro) over ventana_3 as avg3_mcaja_ahorro
#     , avg(mcaja_ahorro_adicional) over ventana_3 as avg3_mcaja_ahorro_adicional
#     , avg(mcaja_ahorro_dolares) over ventana_3 as avg3_mcaja_ahorro_dolares
#     , avg(mcuentas_saldo) over ventana_3 as avg3_mcuentas_saldo
#     , avg(ctarjeta_debito) over ventana_3 as avg3_ctarjeta_debito
#     , avg(ctarjeta_debito_transacciones) over ventana_3 as avg3_ctarjeta_debito_transacciones
#     , avg(mautoservicio) over ventana_3 as avg3_mautoservicio
#     , avg(ctarjeta_visa) over ventana_3 as avg3_ctarjeta_visa
#     , avg(ctarjeta_visa_transacciones) over ventana_3 as avg3_ctarjeta_visa_transacciones
#     , avg(mtarjeta_visa_consumo) over ventana_3 as avg3_mtarjeta_visa_consumo
#     , avg(ctarjeta_master) over ventana_3 as avg3_ctarjeta_master
#     , avg(ctarjeta_master_transacciones) over ventana_3 as avg3_ctarjeta_master_transacciones
#     , avg(mtarjeta_master_consumo) over ventana_3 as avg3_mtarjeta_master_consumo
#     , avg(cprestamos_prendarios) over ventana_3 as avg3_cprestamos_prendarios
#     , avg(mprestamos_prendarios) over ventana_3 as avg3_mprestamos_prendarios
#     , avg(cprestamos_hipotecarios) over ventana_3 as avg3_cprestamos_hipotecarios
#     , avg(mprestamos_hipotecarios) over ventana_3 as avg3_mprestamos_hipotecarios
#     , avg(cplazo_fijo) over ventana_3 as avg3_cplazo_fijo
#     , avg(mplazo_fijo_dolares) over ventana_3 as avg3_mplazo_fijo_dolares
#     , avg(mplazo_fijo_pesos) over ventana_3 as avg3_mplazo_fijo_pesos
#     , avg(cinversion1) over ventana_3 as avg3_cinversion1
#     , avg(minversion1_pesos) over ventana_3 as avg3_minversion1_pesos
#     , avg(minversion1_dolares) over ventana_3 as avg3_minversion1_dolares
#     , avg(cinversion2) over ventana_3 as avg3_cinversion2
#     , avg(minversion2) over ventana_3 as avg3_minversion2
#     , avg(cseguro_vida) over ventana_3 as avg3_cseguro_vida
#     , avg(cseguro_auto) over ventana_3 as avg3_cseguro_auto
#     , avg(cseguro_vivienda) over ventana_3 as avg3_cseguro_vivienda
#     , avg(cseguro_accidentes_personales) over ventana_3 as avg3_cseguro_accidentes_personales
#     , avg(cpayroll_trx) over ventana_3 as avg3_cpayroll_trx
#     , avg(mpayroll) over ventana_3 as avg3_mpayroll
#     , avg(mpayroll2) over ventana_3 as avg3_mpayroll2
#     , avg(cpayroll2_trx) over ventana_3 as avg3_cpayroll2_trx
#     , avg(ccuenta_debitos_automaticos) over ventana_3 as avg3_ccuenta_debitos_automaticos
#     , avg(mcuenta_debitos_automaticos) over ventana_3 as avg3_mcuenta_debitos_automaticos
#     , avg(ctarjeta_visa_debitos_automaticos) over ventana_3 as avg3_ctarjeta_visa_debitos_automaticos
#     , avg(mttarjeta_visa_debitos_automaticos) over ventana_3 as avg3_mttarjeta_visa_debitos_automaticos
#     , avg(ctarjeta_master_debitos_automaticos) over ventana_3 as avg3_ctarjeta_master_debitos_automaticos
#     , avg(mttarjeta_master_debitos_automaticos) over ventana_3 as avg3_mttarjeta_master_debitos_automaticos
#     , avg(cpagodeservicios) over ventana_3 as avg3_cpagodeservicios
#     , avg(mpagodeservicios) over ventana_3 as avg3_mpagodeservicios
#     , avg(cpagomiscuentas) over ventana_3 as avg3_cpagomiscuentas
#     , avg(mpagomiscuentas) over ventana_3 as avg3_mpagomiscuentas
#     , avg(ccajeros_propios_descuentos) over ventana_3 as avg3_ccajeros_propios_descuentos
#     , avg(mcajeros_propios_descuentos) over ventana_3 as avg3_mcajeros_propios_descuentos
#     , avg(ctarjeta_visa_descuentos) over ventana_3 as avg3_ctarjeta_visa_descuentos
#     , avg(mtarjeta_visa_descuentos) over ventana_3 as avg3_mtarjeta_visa_descuentos
#     , avg(ctarjeta_master_descuentos) over ventana_3 as avg3_ctarjeta_master_descuentos
#     , avg(mtarjeta_master_descuentos) over ventana_3 as avg3_mtarjeta_master_descuentos
#     , avg(ccomisiones_mantenimiento) over ventana_3 as avg3_ccomisiones_mantenimiento
#     , avg(mcomisiones_mantenimiento) over ventana_3 as avg3_mcomisiones_mantenimiento
#     , avg(ccomisiones_otras) over ventana_3 as avg3_ccomisiones_otras
#     , avg(mcomisiones_otras) over ventana_3 as avg3_mcomisiones_otras
#     , avg(cforex) over ventana_3 as avg3_cforex
#     , avg(cforex_buy) over ventana_3 as avg3_cforex_buy
#     , avg(mforex_buy) over ventana_3 as avg3_mforex_buy
#     , avg(cforex_sell) over ventana_3 as avg3_cforex_sell
#     , avg(mforex_sell) over ventana_3 as avg3_mforex_sell
#     , avg(ctransferencias_recibidas) over ventana_3 as avg3_ctransferencias_recibidas
#     , avg(mtransferencias_recibidas) over ventana_3 as avg3_mtransferencias_recibidas
#     , avg(ctransferencias_emitidas) over ventana_3 as avg3_ctransferencias_emitidas
#     , avg(mtransferencias_emitidas) over ventana_3 as avg3_mtransferencias_emitidas
#     , avg(cextraccion_autoservicio) over ventana_3 as avg3_cextraccion_autoservicio
#     , avg(mextraccion_autoservicio) over ventana_3 as avg3_mextraccion_autoservicio
#     , avg(ccheques_depositados) over ventana_3 as avg3_ccheques_depositados
#     , avg(mcheques_depositados) over ventana_3 as avg3_mcheques_depositados
#     , avg(ccheques_emitidos) over ventana_3 as avg3_ccheques_emitidos
#     , avg(mcheques_emitidos) over ventana_3 as avg3_mcheques_emitidos
#     , avg(ccheques_depositados_rechazados) over ventana_3 as avg3_ccheques_depositados_rechazados
#     , avg(mcheques_depositados_rechazados) over ventana_3 as avg3_mcheques_depositados_rechazados
#     , avg(ccheques_emitidos_rechazados) over ventana_3 as avg3_ccheques_emitidos_rechazados
#     , avg(mcheques_emitidos_rechazados) over ventana_3 as avg3_mcheques_emitidos_rechazados
#     , avg(ccallcenter_transacciones) over ventana_3 as avg3_ccallcenter_transacciones
#     , avg(chomebanking_transacciones) over ventana_3 as avg3_chomebanking_transacciones
#     , avg(ccajas_transacciones) over ventana_3 as avg3_ccajas_transacciones
#     , avg(ccajas_consultas) over ventana_3 as avg3_ccajas_consultas
#     , avg(ccajas_depositos) over ventana_3 as avg3_ccajas_depositos
#     , avg(ccajas_extracciones) over ventana_3 as avg3_ccajas_extracciones
#     , avg(ccajas_otras) over ventana_3 as avg3_ccajas_otras
#     , avg(catm_trx) over ventana_3 as avg3_catm_trx
#     , avg(matm) over ventana_3 as avg3_matm
#     , avg(catm_trx_other) over ventana_3 as avg3_catm_trx_other
#     , avg(matm_other) over ventana_3 as avg3_matm_other
#     , avg(ctrx_quarter) over ventana_3 as avg3_ctrx_quarter
#     , avg(Master_mfinanciacion_limite) over ventana_3 as avg3_Master_mfinanciacion_limite
#     , avg(Master_msaldototal) over ventana_3 as avg3_Master_msaldototal
#     , avg(Master_msaldopesos) over ventana_3 as avg3_Master_msaldopesos
#     , avg(Master_msaldodolares) over ventana_3 as avg3_Master_msaldodolares
#     , avg(Master_mconsumospesos) over ventana_3 as avg3_Master_mconsumospesos
#     , avg(Master_mconsumosdolares) over ventana_3 as avg3_Master_mconsumosdolares
#     , avg(Master_mlimitecompra) over ventana_3 as avg3_Master_mlimitecompra
#     , avg(Master_madelantopesos) over ventana_3 as avg3_Master_madelantopesos
#     , avg(Master_madelantodolares) over ventana_3 as avg3_Master_madelantodolares
#     , avg(Master_mpagado) over ventana_3 as avg3_Master_mpagado
#     , avg(Master_mpagospesos) over ventana_3 as avg3_Master_mpagospesos
#     , avg(Master_mpagosdolares) over ventana_3 as avg3_Master_mpagosdolares
#     , avg(Master_mconsumototal) over ventana_3 as avg3_Master_mconsumototal
#     , avg(Master_cconsumos) over ventana_3 as avg3_Master_cconsumos
#     , avg(Master_cadelantosefectivo) over ventana_3 as avg3_Master_cadelantosefectivo
#     , avg(Master_mpagominimo) over ventana_3 as avg3_Master_mpagominimo
#     , avg(Visa_mfinanciacion_limite) over ventana_3 as avg3_Visa_mfinanciacion_limite
#     , avg(Visa_msaldototal) over ventana_3 as avg3_Visa_msaldototal
#     , avg(Visa_msaldopesos) over ventana_3 as avg3_Visa_msaldopesos
#     , avg(Visa_msaldodolares) over ventana_3 as avg3_Visa_msaldodolares
#     , avg(Visa_mconsumospesos) over ventana_3 as avg3_Visa_mconsumospesos
#     , avg(Visa_mconsumosdolares) over ventana_3 as avg3_Visa_mconsumosdolares
#     , avg(Visa_mlimitecompra) over ventana_3 as avg3_Visa_mlimitecompra
#     , avg(Visa_madelantopesos) over ventana_3 as avg3_Visa_madelantopesos
#     , avg(Visa_madelantodolares) over ventana_3 as avg3_Visa_madelantodolares
#     , avg(Visa_mpagado) over ventana_3 as avg3_Visa_mpagado
#     , avg(Visa_mpagospesos) over ventana_3 as avg3_Visa_mpagospesos
#     , avg(Visa_mpagosdolares) over ventana_3 as avg3_Visa_mpagosdolares
#     , avg(Visa_mconsumototal) over ventana_3 as avg3_Visa_mconsumototal
#     , avg(Visa_cconsumos) over ventana_3 as avg3_Visa_cconsumos
#     , avg(Visa_cadelantosefectivo) over ventana_3 as avg3_Visa_cadelantosefectivo
#     , avg(Visa_mpagominimo) over ventana_3 as avg3_Visa_mpagominimo
# from competencia_03
# window ventana_3 as (partition by numero_de_cliente order by foto_mes rows between 2 preceding and current row)

,Success


In [ ]:
# %%sql
# create or replace table competencia_03 as
# select
#     *
#     , avg(mrentabilidad) over ventana_12 as avg12_mrentabilidad
#     , avg(mrentabilidad_annual) over ventana_12 as avg12_mrentabilidad_annual
#     , avg(mcomisiones) over ventana_12 as avg12_mcomisiones
#     , avg(mactivos_margen) over ventana_12 as avg12_mactivos_margen
#     , avg(mpasivos_margen) over ventana_12 as avg12_mpasivos_margen
#     , avg(cproductos) over ventana_12 as avg12_cproductos
#     , avg(tcuentas) over ventana_12 as avg12_tcuentas
#     , avg(ccuenta_corriente) over ventana_12 as avg12_ccuenta_corriente
#     , avg(mcuenta_corriente_adicional) over ventana_12 as avg12_mcuenta_corriente_adicional
#     , avg(mcuenta_corriente) over ventana_12 as avg12_mcuenta_corriente
#     , avg(ccaja_ahorro) over ventana_12 as avg12_ccaja_ahorro
#     , avg(mcaja_ahorro) over ventana_12 as avg12_mcaja_ahorro
#     , avg(mcaja_ahorro_adicional) over ventana_12 as avg12_mcaja_ahorro_adicional
#     , avg(mcaja_ahorro_dolares) over ventana_12 as avg12_mcaja_ahorro_dolares
#     , avg(mcuentas_saldo) over ventana_12 as avg12_mcuentas_saldo
#     , avg(ctarjeta_debito) over ventana_12 as avg12_ctarjeta_debito
#     , avg(ctarjeta_debito_transacciones) over ventana_12 as avg12_ctarjeta_debito_transacciones
#     , avg(mautoservicio) over ventana_12 as avg12_mautoservicio
#     , avg(ctarjeta_visa) over ventana_12 as avg12_ctarjeta_visa
#     , avg(ctarjeta_visa_transacciones) over ventana_12 as avg12_ctarjeta_visa_transacciones
#     , avg(mtarjeta_visa_consumo) over ventana_12 as avg12_mtarjeta_visa_consumo
#     , avg(ctarjeta_master) over ventana_12 as avg12_ctarjeta_master
#     , avg(ctarjeta_master_transacciones) over ventana_12 as avg12_ctarjeta_master_transacciones
#     , avg(mtarjeta_master_consumo) over ventana_12 as avg12_mtarjeta_master_consumo
#     , avg(cprestamos_prendarios) over ventana_12 as avg12_cprestamos_prendarios
#     , avg(mprestamos_prendarios) over ventana_12 as avg12_mprestamos_prendarios
#     , avg(cprestamos_hipotecarios) over ventana_12 as avg12_cprestamos_hipotecarios
#     , avg(mprestamos_hipotecarios) over ventana_12 as avg12_mprestamos_hipotecarios
#     , avg(cplazo_fijo) over ventana_12 as avg12_cplazo_fijo
#     , avg(mplazo_fijo_dolares) over ventana_12 as avg12_mplazo_fijo_dolares
#     , avg(mplazo_fijo_pesos) over ventana_12 as avg12_mplazo_fijo_pesos
#     , avg(cinversion1) over ventana_12 as avg12_cinversion1
#     , avg(minversion1_pesos) over ventana_12 as avg12_minversion1_pesos
#     , avg(minversion1_dolares) over ventana_12 as avg12_minversion1_dolares
#     , avg(cinversion2) over ventana_12 as avg12_cinversion2
#     , avg(minversion2) over ventana_12 as avg12_minversion2
#     , avg(cseguro_vida) over ventana_12 as avg12_cseguro_vida
#     , avg(cseguro_auto) over ventana_12 as avg12_cseguro_auto
#     , avg(cseguro_vivienda) over ventana_12 as avg12_cseguro_vivienda
#     , avg(cseguro_accidentes_personales) over ventana_12 as avg12_cseguro_accidentes_personales
#     , avg(cpayroll_trx) over ventana_12 as avg12_cpayroll_trx
#     , avg(mpayroll) over ventana_12 as avg12_mpayroll
#     , avg(mpayroll2) over ventana_12 as avg12_mpayroll2
#     , avg(cpayroll2_trx) over ventana_12 as avg12_cpayroll2_trx
#     , avg(ccuenta_debitos_automaticos) over ventana_12 as avg12_ccuenta_debitos_automaticos
#     , avg(mcuenta_debitos_automaticos) over ventana_12 as avg12_mcuenta_debitos_automaticos
#     , avg(ctarjeta_visa_debitos_automaticos) over ventana_12 as avg12_ctarjeta_visa_debitos_automaticos
#     , avg(mttarjeta_visa_debitos_automaticos) over ventana_12 as avg12_mttarjeta_visa_debitos_automaticos
#     , avg(ctarjeta_master_debitos_automaticos) over ventana_12 as avg12_ctarjeta_master_debitos_automaticos
#     , avg(mttarjeta_master_debitos_automaticos) over ventana_12 as avg12_mttarjeta_master_debitos_automaticos
#     , avg(cpagodeservicios) over ventana_12 as avg12_cpagodeservicios
#     , avg(mpagodeservicios) over ventana_12 as avg12_mpagodeservicios
#     , avg(cpagomiscuentas) over ventana_12 as avg12_cpagomiscuentas
#     , avg(mpagomiscuentas) over ventana_12 as avg12_mpagomiscuentas
#     , avg(ccajeros_propios_descuentos) over ventana_12 as avg12_ccajeros_propios_descuentos
#     , avg(mcajeros_propios_descuentos) over ventana_12 as avg12_mcajeros_propios_descuentos
#     , avg(ctarjeta_visa_descuentos) over ventana_12 as avg12_ctarjeta_visa_descuentos
#     , avg(mtarjeta_visa_descuentos) over ventana_12 as avg12_mtarjeta_visa_descuentos
#     , avg(ctarjeta_master_descuentos) over ventana_12 as avg12_ctarjeta_master_descuentos
#     , avg(mtarjeta_master_descuentos) over ventana_12 as avg12_mtarjeta_master_descuentos
#     , avg(ccomisiones_mantenimiento) over ventana_12 as avg12_ccomisiones_mantenimiento
#     , avg(mcomisiones_mantenimiento) over ventana_12 as avg12_mcomisiones_mantenimiento
#     , avg(ccomisiones_otras) over ventana_12 as avg12_ccomisiones_otras
#     , avg(mcomisiones_otras) over ventana_12 as avg12_mcomisiones_otras
#     , avg(cforex) over ventana_12 as avg12_cforex
#     , avg(cforex_buy) over ventana_12 as avg12_cforex_buy
#     , avg(mforex_buy) over ventana_12 as avg12_mforex_buy
#     , avg(cforex_sell) over ventana_12 as avg12_cforex_sell
#     , avg(mforex_sell) over ventana_12 as avg12_mforex_sell
#     , avg(ctransferencias_recibidas) over ventana_12 as avg12_ctransferencias_recibidas
#     , avg(mtransferencias_recibidas) over ventana_12 as avg12_mtransferencias_recibidas
#     , avg(ctransferencias_emitidas) over ventana_12 as avg12_ctransferencias_emitidas
#     , avg(mtransferencias_emitidas) over ventana_12 as avg12_mtransferencias_emitidas
#     , avg(cextraccion_autoservicio) over ventana_12 as avg12_cextraccion_autoservicio
#     , avg(mextraccion_autoservicio) over ventana_12 as avg12_mextraccion_autoservicio
#     , avg(ccheques_depositados) over ventana_12 as avg12_ccheques_depositados
#     , avg(mcheques_depositados) over ventana_12 as avg12_mcheques_depositados
#     , avg(ccheques_emitidos) over ventana_12 as avg12_ccheques_emitidos
#     , avg(mcheques_emitidos) over ventana_12 as avg12_mcheques_emitidos
#     , avg(ccheques_depositados_rechazados) over ventana_12 as avg12_ccheques_depositados_rechazados
#     , avg(mcheques_depositados_rechazados) over ventana_12 as avg12_mcheques_depositados_rechazados
#     , avg(ccheques_emitidos_rechazados) over ventana_12 as avg12_ccheques_emitidos_rechazados
#     , avg(mcheques_emitidos_rechazados) over ventana_12 as avg12_mcheques_emitidos_rechazados
#     , avg(ccallcenter_transacciones) over ventana_12 as avg12_ccallcenter_transacciones
#     , avg(chomebanking_transacciones) over ventana_12 as avg12_chomebanking_transacciones
#     , avg(ccajas_transacciones) over ventana_12 as avg12_ccajas_transacciones
#     , avg(ccajas_consultas) over ventana_12 as avg12_ccajas_consultas
#     , avg(ccajas_depositos) over ventana_12 as avg12_ccajas_depositos
#     , avg(ccajas_extracciones) over ventana_12 as avg12_ccajas_extracciones
#     , avg(ccajas_otras) over ventana_12 as avg12_ccajas_otras
#     , avg(catm_trx) over ventana_12 as avg12_catm_trx
#     , avg(matm) over ventana_12 as avg12_matm
#     , avg(catm_trx_other) over ventana_12 as avg12_catm_trx_other
#     , avg(matm_other) over ventana_12 as avg12_matm_other
#     , avg(ctrx_quarter) over ventana_12 as avg12_ctrx_quarter
#     , avg(Master_mfinanciacion_limite) over ventana_12 as avg12_Master_mfinanciacion_limite
#     , avg(Master_msaldototal) over ventana_12 as avg12_Master_msaldototal
#     , avg(Master_msaldopesos) over ventana_12 as avg12_Master_msaldopesos
#     , avg(Master_msaldodolares) over ventana_12 as avg12_Master_msaldodolares
#     , avg(Master_mconsumospesos) over ventana_12 as avg12_Master_mconsumospesos
#     , avg(Master_mconsumosdolares) over ventana_12 as avg12_Master_mconsumosdolares
#     , avg(Master_mlimitecompra) over ventana_12 as avg12_Master_mlimitecompra
#     , avg(Master_madelantopesos) over ventana_12 as avg12_Master_madelantopesos
#     , avg(Master_madelantodolares) over ventana_12 as avg12_Master_madelantodolares
#     , avg(Master_mpagado) over ventana_12 as avg12_Master_mpagado
#     , avg(Master_mpagospesos) over ventana_12 as avg12_Master_mpagospesos
#     , avg(Master_mpagosdolares) over ventana_12 as avg12_Master_mpagosdolares
#     , avg(Master_mconsumototal) over ventana_12 as avg12_Master_mconsumototal
#     , avg(Master_cconsumos) over ventana_12 as avg12_Master_cconsumos
#     , avg(Master_cadelantosefectivo) over ventana_12 as avg12_Master_cadelantosefectivo
#     , avg(Master_mpagominimo) over ventana_12 as avg12_Master_mpagominimo
#     , avg(Visa_mfinanciacion_limite) over ventana_12 as avg12_Visa_mfinanciacion_limite
#     , avg(Visa_msaldototal) over ventana_12 as avg12_Visa_msaldototal
#     , avg(Visa_msaldopesos) over ventana_12 as avg12_Visa_msaldopesos
#     , avg(Visa_msaldodolares) over ventana_12 as avg12_Visa_msaldodolares
#     , avg(Visa_mconsumospesos) over ventana_12 as avg12_Visa_mconsumospesos
#     , avg(Visa_mconsumosdolares) over ventana_12 as avg12_Visa_mconsumosdolares
#     , avg(Visa_mlimitecompra) over ventana_12 as avg12_Visa_mlimitecompra
#     , avg(Visa_madelantopesos) over ventana_12 as avg12_Visa_madelantopesos
#     , avg(Visa_madelantodolares) over ventana_12 as avg12_Visa_madelantodolares
#     , avg(Visa_mpagado) over ventana_12 as avg12_Visa_mpagado
#     , avg(Visa_mpagospesos) over ventana_12 as avg12_Visa_mpagospesos
#     , avg(Visa_mpagosdolares) over ventana_12 as avg12_Visa_mpagosdolares
#     , avg(Visa_mconsumototal) over ventana_12 as avg12_Visa_mconsumototal
#     , avg(Visa_cconsumos) over ventana_12 as avg12_Visa_cconsumos
#     , avg(Visa_cadelantosefectivo) over ventana_12 as avg12_Visa_cadelantosefectivo
#     , avg(Visa_mpagominimo) over ventana_12 as avg12_Visa_mpagominimo
# from competencia_03
# window ventana_12 as (partition by numero_de_cliente order by foto_mes rows between 11 preceding and current row)



Variables creadas

In [ ]:
# %%sql
# create or replace table competencia_03 as
# select
#     *
#     , avg(cc_ca) over ventana_3 as avg3_cc_ca
#     , avg(ctarjeta_total) over ventana_3 as avg3_ctarjeta_total
#     , avg(ctarjeta_total_transacciones) over ventana_3 as avg3_ctarjeta_total_transacciones
#     , avg(mtarjeta_total_consumo) over ventana_3 as avg3_mtarjeta_total_consumo
#     , avg(mplazo_fijo) over ventana_3 as avg3_mplazo_fijo
#     , avg(minversion1) over ventana_3 as avg3_minversion1
#     , avg(cseguro_todos) over ventana_3 as avg3_cseguro_todos
#     , avg(mpayroll_todos) over ventana_3 as avg3_mpayroll_todos
#     , avg(consumos_mas_DA) over ventana_3 as avg3_consumos_mas_DA
#     , avg(ctarjeta_todas_debitos_automaticos) over ventana_3 as avg3_ctarjeta_todas_debitos_automaticos
#     , avg(mtarjeta_todas_debitos_automaticos) over ventana_3 as avg3_mtarjeta_todas_debitos_automaticos
#     , avg(ctarjeta_todas_descuentos) over ventana_3 as avg3_ctarjeta_todas_descuentos
#     , avg(mtarjeta_todas_descuentos) over ventana_3 as avg3_mtarjeta_todas_descuentos
#     , avg(comisiones) over ventana_3 as avg3_comisiones
#     , avg(mcomisiones_suma) over ventana_3 as avg3_mcomisiones_suma
#     , avg(ctransferencias) over ventana_3 as avg3_ctransferencias
#     , avg(mtransferencias) over ventana_3 as avg3_mtransferencias
#     , avg(tarjeta_msaldototal) over ventana_3 as avg3_tarjeta_msaldototal
#     , avg(tarjetas_msaldopesos) over ventana_3 as avg3_tarjetas_msaldopesos
#     , avg(tarjetas_msaldodolares) over ventana_3 as avg3_tarjetas_msaldodolares
#     , avg(tarjetas_mconsumospesos) over ventana_3 as avg3_tarjetas_mconsumospesos
#     , avg(tarjetas_mconsumosdolares) over ventana_3 as avg3_tarjetas_mconsumosdolares
#     , avg(tarjetas_mlimitecompra) over ventana_3 as avg3_tarjetas_mlimitecompra
#     , avg(tarjetas_madelantopesos) over ventana_3 as avg3_tarjetas_madelantopesos
#     , avg(tarjetas_madelantodolares) over ventana_3 as avg3_tarjetas_madelantodolares
#     , avg(tarjetas_mpagado) over ventana_3 as avg3_tarjetas_mpagado
#     , avg(tarjetas_mpagospesos) over ventana_3 as avg3_tarjetas_mpagospesos
#     , avg(tarjetas_mpagosdolares) over ventana_3 as avg3_tarjetas_mpagosdolares
#     , avg(tarjetas_mconsumototal) over ventana_3 as avg3_tarjetas_mconsumototal
#     , avg(tarjetas_cconsumos) over ventana_3 as avg3_tarjetas_cconsumos
#     , avg(tarjetas_cadelantosefectivo) over ventana_3 as avg3_tarjetas_cadelantosefectivo
#     , avg(tarjetas_mpagominimo) over ventana_3 as avg3_tarjetas_mpagominimo
#     , avg(tarjetas_mfinanciacion_limite) over ventana_3 as avg3_tarjetas_mfinanciacion_limite
#     , avg(matm_total) over ventana_3 as avg3_matm_total
# from competencia_03
# window ventana_3 as (partition by numero_de_cliente order by foto_mes rows between 2 preceding and current row)

,Success


In [ ]:
# %%sql
# create or replace table competencia_03 as
# select
#     *
#     , avg(cc_ca) over ventana_12 as avg12_cc_ca
#     , avg(ctarjeta_total) over ventana_12 as avg12_ctarjeta_total
#     , avg(ctarjeta_total_transacciones) over ventana_12 as avg12_ctarjeta_total_transacciones
#     , avg(mtarjeta_total_consumo) over ventana_12 as avg12_mtarjeta_total_consumo
#     , avg(mplazo_fijo) over ventana_12 as avg12_mplazo_fijo
#     , avg(minversion1) over ventana_12 as avg12_minversion1
#     , avg(cseguro_todos) over ventana_12 as avg12_cseguro_todos
#     , avg(mpayroll_todos) over ventana_12 as avg12_mpayroll_todos
#     , avg(consumos_mas_DA) over ventana_12 as avg12_consumos_mas_DA
#     , avg(ctarjeta_todas_debitos_automaticos) over ventana_12 as avg12_ctarjeta_todas_debitos_automaticos
#     , avg(mtarjeta_todas_debitos_automaticos) over ventana_12 as avg12_mtarjeta_todas_debitos_automaticos
#     , avg(ctarjeta_todas_descuentos) over ventana_12 as avg12_ctarjeta_todas_descuentos
#     , avg(mtarjeta_todas_descuentos) over ventana_12 as avg12_mtarjeta_todas_descuentos
#     , avg(comisiones) over ventana_12 as avg12_comisiones
#     , avg(mcomisiones_suma) over ventana_12 as avg12_mcomisiones_suma
#     , avg(ctransferencias) over ventana_12 as avg12_ctransferencias
#     , avg(mtransferencias) over ventana_12 as avg12_mtransferencias
#     , avg(tarjeta_msaldototal) over ventana_12 as avg12_tarjeta_msaldototal
#     , avg(tarjetas_msaldopesos) over ventana_12 as avg12_tarjetas_msaldopesos
#     , avg(tarjetas_msaldodolares) over ventana_12 as avg12_tarjetas_msaldodolares
#     , avg(tarjetas_mconsumospesos) over ventana_12 as avg12_tarjetas_mconsumospesos
#     , avg(tarjetas_mconsumosdolares) over ventana_12 as avg12_tarjetas_mconsumosdolares
#     , avg(tarjetas_mlimitecompra) over ventana_12 as avg12_tarjetas_mlimitecompra
#     , avg(tarjetas_madelantopesos) over ventana_12 as avg12_tarjetas_madelantopesos
#     , avg(tarjetas_madelantodolares) over ventana_12 as avg12_tarjetas_madelantodolares
#     , avg(tarjetas_mpagado) over ventana_12 as avg12_tarjetas_mpagado
#     , avg(tarjetas_mpagospesos) over ventana_12 as avg12_tarjetas_mpagospesos
#     , avg(tarjetas_mpagosdolares) over ventana_12 as avg12_tarjetas_mpagosdolares
#     , avg(tarjetas_mconsumototal) over ventana_12 as avg12_tarjetas_mconsumototal
#     , avg(tarjetas_cconsumos) over ventana_12 as avg12_tarjetas_cconsumos
#     , avg(tarjetas_cadelantosefectivo) over ventana_12 as avg12_tarjetas_cadelantosefectivo
#     , avg(tarjetas_mpagominimo) over ventana_12 as avg12_tarjetas_mpagominimo
#     , avg(tarjetas_mfinanciacion_limite) over ventana_12 as avg12_tarjetas_mfinanciacion_limite
#     , avg(matm_total) over ventana_12 as avg12_matm_total
# from competencia_03
# window ventana_12 as (partition by numero_de_cliente order by foto_mes rows between 11 preceding and current row)


### Slope

Variables originales

In [ ]:
%%sql
create or replace table competencia_03 as
select
    *
    , regr_slope(mrentabilidad, cliente_antiguedad) over ventana_3 as slope_mrentabilidad
    , regr_slope(mrentabilidad_annual, cliente_antiguedad) over ventana_3 as slope_mrentabilidad_annual
    , regr_slope(mcomisiones, cliente_antiguedad) over ventana_3 as slope_mcomisiones
    , regr_slope(mactivos_margen, cliente_antiguedad) over ventana_3 as slope_mactivos_margen
    , regr_slope(mpasivos_margen, cliente_antiguedad) over ventana_3 as slope_mpasivos_margen
    , regr_slope(cproductos, cliente_antiguedad) over ventana_3 as slope_cproductos
    , regr_slope(tcuentas, cliente_antiguedad) over ventana_3 as slope_tcuentas
    , regr_slope(ccuenta_corriente, cliente_antiguedad) over ventana_3 as slope_ccuenta_corriente
    , regr_slope(mcuenta_corriente_adicional, cliente_antiguedad) over ventana_3 as slope_mcuenta_corriente_adicional
    , regr_slope(mcuenta_corriente, cliente_antiguedad) over ventana_3 as slope_mcuenta_corriente
    , regr_slope(ccaja_ahorro, cliente_antiguedad) over ventana_3 as slope_ccaja_ahorro
    , regr_slope(mcaja_ahorro, cliente_antiguedad) over ventana_3 as slope_mcaja_ahorro
    , regr_slope(mcaja_ahorro_adicional, cliente_antiguedad) over ventana_3 as slope_mcaja_ahorro_adicional
    , regr_slope(mcaja_ahorro_dolares, cliente_antiguedad) over ventana_3 as slope_mcaja_ahorro_dolares
    , regr_slope(mcuentas_saldo, cliente_antiguedad) over ventana_3 as slope_mcuentas_saldo
    , regr_slope(ctarjeta_debito, cliente_antiguedad) over ventana_3 as slope_ctarjeta_debito
    , regr_slope(ctarjeta_debito_transacciones, cliente_antiguedad) over ventana_3 as slope_ctarjeta_debito_transacciones
    , regr_slope(mautoservicio, cliente_antiguedad) over ventana_3 as slope_mautoservicio
    , regr_slope(ctarjeta_visa, cliente_antiguedad) over ventana_3 as slope_ctarjeta_visa
    , regr_slope(ctarjeta_visa_transacciones, cliente_antiguedad) over ventana_3 as slope_ctarjeta_visa_transacciones
    , regr_slope(mtarjeta_visa_consumo, cliente_antiguedad) over ventana_3 as slope_mtarjeta_visa_consumo
    , regr_slope(ctarjeta_master, cliente_antiguedad) over ventana_3 as slope_ctarjeta_master
    , regr_slope(ctarjeta_master_transacciones, cliente_antiguedad) over ventana_3 as slope_ctarjeta_master_transacciones
    , regr_slope(mtarjeta_master_consumo, cliente_antiguedad) over ventana_3 as slope_mtarjeta_master_consumo 
    , regr_slope(cprestamos_prendarios, cliente_antiguedad) over ventana_3 as slope_cprestamos_prendarios
    , regr_slope(mprestamos_prendarios, cliente_antiguedad) over ventana_3 as slope_mprestamos_prendarios
    , regr_slope(cprestamos_hipotecarios, cliente_antiguedad) over ventana_3 as slope_cprestamos_hipotecarios
    , regr_slope(mprestamos_hipotecarios, cliente_antiguedad) over ventana_3 as slope_mprestamos_hipotecarios
    , regr_slope(cplazo_fijo, cliente_antiguedad) over ventana_3 as slope_cplazo_fijo
    , regr_slope(mplazo_fijo_dolares, cliente_antiguedad) over ventana_3 as slope_mplazo_fijo_dolares
    , regr_slope(mplazo_fijo_pesos, cliente_antiguedad) over ventana_3 as slope_mplazo_fijo_pesos
    , regr_slope(cinversion1, cliente_antiguedad) over ventana_3 as slope_cinversion1
    , regr_slope(minversion1_pesos, cliente_antiguedad) over ventana_3 as slope_minversion1_pesos
    , regr_slope(minversion1_dolares, cliente_antiguedad) over ventana_3 as slope_minversion1_dolares
    , regr_slope(cinversion2, cliente_antiguedad) over ventana_3 as slope_cinversion2
    , regr_slope(minversion2, cliente_antiguedad) over ventana_3 as slope_minversion2
    , regr_slope(cseguro_vida, cliente_antiguedad) over ventana_3 as slope_cseguro_vida
    , regr_slope(cseguro_auto, cliente_antiguedad) over ventana_3 as slope_cseguro_auto
    , regr_slope(cseguro_vivienda, cliente_antiguedad) over ventana_3 as slope_cseguro_vivienda
    , regr_slope(cseguro_accidentes_personales, cliente_antiguedad) over ventana_3 as slope_cseguro_accidentes_personales
    , regr_slope(cpayroll_trx, cliente_antiguedad) over ventana_3 as slope_cpayroll_trx
    , regr_slope(mpayroll, cliente_antiguedad) over ventana_3 as slope_mpayroll
    , regr_slope(mpayroll2, cliente_antiguedad) over ventana_3 as slope_mpayroll2
    , regr_slope(cpayroll2_trx, cliente_antiguedad) over ventana_3 as slope_cpayroll2_trx
    , regr_slope(ccuenta_debitos_automaticos, cliente_antiguedad) over ventana_3 as slope_ccuenta_debitos_automaticos
    , regr_slope(mcuenta_debitos_automaticos, cliente_antiguedad) over ventana_3 as slope_mcuenta_debitos_automaticos
    , regr_slope(ctarjeta_visa_debitos_automaticos, cliente_antiguedad) over ventana_3 as slope_ctarjeta_visa_debitos_automaticos
    , regr_slope(mttarjeta_visa_debitos_automaticos, cliente_antiguedad) over ventana_3 as slope_mttarjeta_visa_debitos_automaticos
    , regr_slope(ctarjeta_master_debitos_automaticos, cliente_antiguedad) over ventana_3 as slope_ctarjeta_master_debitos_automaticos
    , regr_slope(mttarjeta_master_debitos_automaticos, cliente_antiguedad) over ventana_3 as slope_mttarjeta_master_debitos_automaticos
    , regr_slope(cpagodeservicios, cliente_antiguedad) over ventana_3 as slope_cpagodeservicios
    , regr_slope(mpagodeservicios, cliente_antiguedad) over ventana_3 as slope_mpagodeservicios
    , regr_slope(cpagomiscuentas, cliente_antiguedad) over ventana_3 as slope_cpagomiscuentas
    , regr_slope(mpagomiscuentas, cliente_antiguedad) over ventana_3 as slope_mpagomiscuentas
    , regr_slope(ccajeros_propios_descuentos, cliente_antiguedad) over ventana_3 as slope_ccajeros_propios_descuentos
    , regr_slope(mcajeros_propios_descuentos, cliente_antiguedad) over ventana_3 as slope_mcajeros_propios_descuentos
    , regr_slope(ctarjeta_visa_descuentos, cliente_antiguedad) over ventana_3 as slope_ctarjeta_visa_descuentos
    , regr_slope(mtarjeta_visa_descuentos, cliente_antiguedad) over ventana_3 as slope_mtarjeta_visa_descuentos
    , regr_slope(ctarjeta_master_descuentos, cliente_antiguedad) over ventana_3 as slope_ctarjeta_master_descuentos
    , regr_slope(mtarjeta_master_descuentos, cliente_antiguedad) over ventana_3 as slope_mtarjeta_master_descuentos
    , regr_slope(ccomisiones_mantenimiento, cliente_antiguedad) over ventana_3 as slope_ccomisiones_mantenimiento
    , regr_slope(mcomisiones_mantenimiento, cliente_antiguedad) over ventana_3 as slope_mcomisiones_mantenimiento
    , regr_slope(ccomisiones_otras, cliente_antiguedad) over ventana_3 as slope_ccomisiones_otras
    , regr_slope(mcomisiones_otras, cliente_antiguedad) over ventana_3 as slope_mcomisiones_otras
    , regr_slope(cforex, cliente_antiguedad) over ventana_3 as slope_cforex
    , regr_slope(cforex_buy, cliente_antiguedad) over ventana_3 as slope_cforex_buy
    , regr_slope(mforex_buy, cliente_antiguedad) over ventana_3 as slope_mforex_buy
    , regr_slope(cforex_sell, cliente_antiguedad) over ventana_3 as slope_cforex_sell
    , regr_slope(mforex_sell, cliente_antiguedad) over ventana_3 as slope_mforex_sell
    , regr_slope(ctransferencias_recibidas, cliente_antiguedad) over ventana_3 as slope_ctransferencias_recibidas
    , regr_slope(mtransferencias_recibidas, cliente_antiguedad) over ventana_3 as slope_mtransferencias_recibidas
    , regr_slope(ctransferencias_emitidas, cliente_antiguedad) over ventana_3 as slope_ctransferencias_emitidas
    , regr_slope(mtransferencias_emitidas, cliente_antiguedad) over ventana_3 as slope_mtransferencias_emitidas
    , regr_slope(cextraccion_autoservicio, cliente_antiguedad) over ventana_3 as slope_cextraccion_autoservicio
    , regr_slope(mextraccion_autoservicio, cliente_antiguedad) over ventana_3 as slope_mextraccion_autoservicio
    , regr_slope(ccheques_depositados, cliente_antiguedad) over ventana_3 as slope_ccheques_depositados
    , regr_slope(mcheques_depositados, cliente_antiguedad) over ventana_3 as slope_mcheques_depositados
    , regr_slope(ccheques_emitidos, cliente_antiguedad) over ventana_3 as slope_ccheques_emitidos
    , regr_slope(mcheques_emitidos, cliente_antiguedad) over ventana_3 as slope_mcheques_emitidos
    , regr_slope(ccheques_depositados_rechazados, cliente_antiguedad) over ventana_3 as slope_ccheques_depositados_rechazados
    , regr_slope(mcheques_depositados_rechazados, cliente_antiguedad) over ventana_3 as slope_mcheques_depositados_rechazados
    , regr_slope(ccheques_emitidos_rechazados, cliente_antiguedad) over ventana_3 as slope_ccheques_emitidos_rechazados
    , regr_slope(mcheques_emitidos_rechazados, cliente_antiguedad) over ventana_3 as slope_mcheques_emitidos_rechazados
    , regr_slope(ccallcenter_transacciones, cliente_antiguedad) over ventana_3 as slope_ccallcenter_transacciones
    , regr_slope(chomebanking_transacciones, cliente_antiguedad) over ventana_3 as slope_chomebanking_transacciones
    , regr_slope(ccajas_transacciones, cliente_antiguedad) over ventana_3 as slope_ccajas_transacciones
    , regr_slope(ccajas_consultas, cliente_antiguedad) over ventana_3 as slope_ccajas_consultas
    , regr_slope(ccajas_depositos, cliente_antiguedad) over ventana_3 as slope_ccajas_depositos
    , regr_slope(ccajas_extracciones, cliente_antiguedad) over ventana_3 as slope_ccajas_extracciones
    , regr_slope(ccajas_otras, cliente_antiguedad) over ventana_3 as slope_ccajas_otras
    , regr_slope(catm_trx, cliente_antiguedad) over ventana_3 as slope_catm_trx
    , regr_slope(matm, cliente_antiguedad) over ventana_3 as slope_matm
    , regr_slope(catm_trx_other, cliente_antiguedad) over ventana_3 as slope_catm_trx_other
    , regr_slope(matm_other, cliente_antiguedad) over ventana_3 as slope_matm_other
    , regr_slope(ctrx_quarter, cliente_antiguedad) over ventana_3 as slope_ctrx_quarter
    , regr_slope(Master_mfinanciacion_limite, cliente_antiguedad) over ventana_3 as slope_Master_mfinanciacion_limite
    , regr_slope(Master_msaldototal, cliente_antiguedad) over ventana_3 as slope_Master_msaldototal
    , regr_slope(Master_msaldopesos, cliente_antiguedad) over ventana_3 as slope_Master_msaldopesos
    , regr_slope(Master_msaldodolares, cliente_antiguedad) over ventana_3 as slope_Master_msaldodolares
    , regr_slope(Master_mconsumospesos, cliente_antiguedad) over ventana_3 as slope_Master_mconsumospesos
    , regr_slope(Master_mconsumosdolares, cliente_antiguedad) over ventana_3 as slope_Master_mconsumosdolares
    , regr_slope(Master_mlimitecompra, cliente_antiguedad) over ventana_3 as slope_Master_mlimitecompra
    , regr_slope(Master_madelantopesos, cliente_antiguedad) over ventana_3 as slope_Master_madelantopesos
    , regr_slope(Master_madelantodolares, cliente_antiguedad) over ventana_3 as slope_Master_madelantodolares
    , regr_slope(Master_mpagado, cliente_antiguedad) over ventana_3 as slope_Master_mpagado
    , regr_slope(Master_mpagospesos, cliente_antiguedad) over ventana_3 as slope_Master_mpagospesos
    , regr_slope(Master_mpagosdolares, cliente_antiguedad) over ventana_3 as slope_Master_mpagosdolares
    , regr_slope(Master_mconsumototal, cliente_antiguedad) over ventana_3 as slope_Master_mconsumototal
    , regr_slope(Master_cconsumos, cliente_antiguedad) over ventana_3 as slope_Master_cconsumos
    , regr_slope(Master_cadelantosefectivo, cliente_antiguedad) over ventana_3 as slope_Master_cadelantosefectivo
    , regr_slope(Master_mpagominimo, cliente_antiguedad) over ventana_3 as slope_Master_mpagominimo
    , regr_slope(Visa_mfinanciacion_limite, cliente_antiguedad) over ventana_3 as slope_Visa_mfinanciacion_limite
    , regr_slope(Visa_msaldototal, cliente_antiguedad) over ventana_3 as slope_Visa_msaldototal
    , regr_slope(Visa_msaldopesos, cliente_antiguedad) over ventana_3 as slope_Visa_msaldopesos
    , regr_slope(Visa_msaldodolares, cliente_antiguedad) over ventana_3 as slope_Visa_msaldodolares
    , regr_slope(Visa_mconsumospesos, cliente_antiguedad) over ventana_3 as slope_Visa_mconsumospesos
    , regr_slope(Visa_mconsumosdolares, cliente_antiguedad) over ventana_3 as slope_Visa_mconsumosdolares
    , regr_slope(Visa_mlimitecompra, cliente_antiguedad) over ventana_3 as slope_Visa_mlimitecompra
    , regr_slope(Visa_madelantopesos, cliente_antiguedad) over ventana_3 as slope_Visa_madelantopesos
    , regr_slope(Visa_madelantodolares, cliente_antiguedad) over ventana_3 as slope_Visa_madelantodolares
    , regr_slope(Visa_mpagado, cliente_antiguedad) over ventana_3 as slope_Visa_mpagado
    , regr_slope(Visa_mpagospesos, cliente_antiguedad) over ventana_3 as slope_Visa_mpagospesos
    , regr_slope(Visa_mpagosdolares, cliente_antiguedad) over ventana_3 as slope_Visa_mpagosdolares
    , regr_slope(Visa_mconsumototal, cliente_antiguedad) over ventana_3 as slope_Visa_mconsumototal
    , regr_slope(Visa_cconsumos, cliente_antiguedad) over ventana_3 as slope_Visa_cconsumos
    , regr_slope(Visa_cadelantosefectivo, cliente_antiguedad) over ventana_3 as slope_Visa_cadelantosefectivo
    , regr_slope(Visa_mpagominimo, cliente_antiguedad) over ventana_3 as slope_Visa_mpagominimo
from competencia_03
window ventana_3 as (partition by numero_de_cliente order by foto_mes rows between 2 preceding and current row)


,Success


Variables creadas

In [ ]:
# esto no estaba comentado, lo comento yo 

%%sql
create or replace table competencia_03 as
select
    *
    , regr_slope(cc_ca, cliente_antiguedad) over ventana_3 as slope_cc_ca
    , regr_slope(ctarjeta_total, cliente_antiguedad) over ventana_3 as slope_ctarjeta_total
    , regr_slope(ctarjeta_total_transacciones, cliente_antiguedad) over ventana_3 as slope_ctarjeta_total_transacciones
    , regr_slope(mtarjeta_total_consumo, cliente_antiguedad) over ventana_3 as slope_mtarjeta_total_consumo
    , regr_slope(mplazo_fijo, cliente_antiguedad) over ventana_3 as slope_mplazo_fijo
    , regr_slope(minversion1, cliente_antiguedad) over ventana_3 as slope_minversion1
    , regr_slope(cseguro_todos, cliente_antiguedad) over ventana_3 as slope_cseguro_todos
    , regr_slope(mpayroll_todos, cliente_antiguedad) over ventana_3 as slope_mpayroll_todos
    , regr_slope(consumos_mas_DA, cliente_antiguedad) over ventana_3 as slope_consumos_mas_DA
    , regr_slope(ctarjeta_todas_debitos_automaticos, cliente_antiguedad) over ventana_3 as slope_ctarjeta_todas_debitos_automaticos
    , regr_slope(mtarjeta_todas_debitos_automaticos, cliente_antiguedad) over ventana_3 as slope_mtarjeta_todas_debitos_automaticos
    , regr_slope(ctarjeta_todas_descuentos, cliente_antiguedad) over ventana_3 as slope_ctarjeta_todas_descuentos
    , regr_slope(mtarjeta_todas_descuentos, cliente_antiguedad) over ventana_3 as slope_mtarjeta_todas_descuentos
    , regr_slope(comisiones, cliente_antiguedad) over ventana_3 as slope_comisiones
    , regr_slope(mcomisiones_suma, cliente_antiguedad) over ventana_3 as slope_mcomisiones_suma
    , regr_slope(ctransferencias, cliente_antiguedad) over ventana_3 as slope_ctransferencias
    , regr_slope(mtransferencias, cliente_antiguedad) over ventana_3 as slope_mtransferencias
    , regr_slope(tarjeta_msaldototal, cliente_antiguedad) over ventana_3 as slope_tarjeta_msaldototal
    , regr_slope(tarjetas_msaldopesos, cliente_antiguedad) over ventana_3 as slope_tarjetas_msaldopesos
    , regr_slope(tarjetas_msaldodolares, cliente_antiguedad) over ventana_3 as slope_tarjetas_msaldodolares
    , regr_slope(tarjetas_mconsumospesos, cliente_antiguedad) over ventana_3 as slope_tarjetas_mconsumospesos
    , regr_slope(tarjetas_mconsumosdolares, cliente_antiguedad) over ventana_3 as slope_tarjetas_mconsumosdolares
    , regr_slope(tarjetas_mlimitecompra, cliente_antiguedad) over ventana_3 as slope_tarjetas_mlimitecompra
    , regr_slope(tarjetas_madelantopesos, cliente_antiguedad) over ventana_3 as slope_tarjetas_madelantopesos
    , regr_slope(tarjetas_madelantodolares, cliente_antiguedad) over ventana_3 as slope_tarjetas_madelantodolares
    , regr_slope(tarjetas_mpagado, cliente_antiguedad) over ventana_3 as slope_tarjetas_mpagado
    , regr_slope(tarjetas_mpagospesos, cliente_antiguedad) over ventana_3 as slope_tarjetas_mpagospesos
    , regr_slope(tarjetas_mpagosdolares, cliente_antiguedad) over ventana_3 as slope_tarjetas_mpagosdolares
    , regr_slope(tarjetas_mconsumototal, cliente_antiguedad) over ventana_3 as slope_tarjetas_mconsumototal
    , regr_slope(tarjetas_cconsumos, cliente_antiguedad) over ventana_3 as slope_tarjetas_cconsumos
    , regr_slope(tarjetas_cadelantosefectivo, cliente_antiguedad) over ventana_3 as slope_tarjetas_cadelantosefectivo
    , regr_slope(tarjetas_mpagominimo, cliente_antiguedad) over ventana_3 as slope_tarjetas_mpagominimo
    , regr_slope(tarjetas_mfinanciacion_limite, cliente_antiguedad) over ventana_3 as slope_tarjetas_mfinanciacion_limite
    , regr_slope(matm_total, cliente_antiguedad) over ventana_3 as slope_matm_total
from competencia_03
window ventana_3 as (partition by numero_de_cliente order by foto_mes rows between 2 preceding and current row)

### Guardar datos

In [ ]:
%%sql 

select * from competencia_03

limit 10

In [ ]:
%sql COPY competencia_03 TO '{{dataset_path}}competencia_03_fe_k300.parquet' (FORMAT PARQUET)

,Success
